In [26]:
import sagemaker
import boto3
import sys
import pandas as pd
import numpy as np 
import io 
from sagemaker.session import Session
from sagemaker import get_execution_role

In [27]:
prefix = "aws-mlops-sagemaker"
role = get_execution_role()

sagemaker_session = Session() 
region = sagemaker_session.boto_region_name
s3_bucket_name = sagemaker_session.default_bucket()

In [28]:
region

'us-east-1'

In [29]:
s3_bucket_name

'sagemaker-us-east-1-866824485776'

In [30]:
role

'arn:aws:iam::866824485776:role/service-role/AmazonSageMaker-ExecutionRole-20240502T094083'

In [31]:
# Data 
customer_data = pd.read_csv("transformed/customers.csv") 
orders_data = pd.read_csv("transformed/orders.csv")

In [32]:
customer_data.head()

,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C1,0,1,2024-05-02T05:39:10.965Z,0,0,0,0,0,1,0.203425
1,C2,1,1,2024-05-02T05:39:10.966Z,0,0,0,0,0,1,0.859589
2,C3,1,1,2024-05-02T05:39:10.967Z,0,1,0,0,0,0,0.527397
3,C4,1,0,2024-05-02T05:39:10.967Z,0,0,0,0,1,0,0.780822
4,C5,1,0,2024-05-02T05:39:10.968Z,0,0,0,0,1,0,0.691096


In [33]:
orders_data.head()

,order_id,customer_id,product_id,purchase_amount,is_reordered,event_time,n_days_since_last_purchase
0,O1,C9765,P11660,0.572673,0,2024-05-02T05:39:17.172Z,0.273256
1,O2,C3674,P6868,0.693861,0,2024-05-02T05:39:17.172Z,0.846899
2,O3,C2139,P4749,0.556139,1,2024-05-02T05:39:17.172Z,0.408915
3,O4,C7794,P542,0.043069,1,2024-05-02T05:39:17.172Z,0.843023
4,O5,C2229,P7605,0.463861,1,2024-05-02T05:39:17.172Z,0.265504


In [34]:
# Create feature group
from time import gmtime, strftime, sleep

customers_feature_group_name = "customers-fg-"+strftime("%d-%H-%M-%S", gmtime()) 
orders_feature_group_name = "orders-fg-"+strftime("%d-%H-%M-%S", gmtime()) 

In [35]:
customers_feature_group_name

'customers-fg-02-13-01-25'

In [36]:
orders_feature_group_name

'orders-fg-02-13-01-25'

In [37]:
from sagemaker.feature_store.feature_group import FeatureGroup

customers_feature_group = FeatureGroup(name=customers_feature_group_name , 
                                       sagemaker_session = sagemaker_session)

orders_feature_group = FeatureGroup(name=orders_feature_group_name , 
                                       sagemaker_session = sagemaker_session)

In [38]:
record_identifier_feature_name = "customer_id"

In [24]:
orders_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   order_id                    100000 non-null  object             
 1   customer_id                 100000 non-null  object             
 2   product_id                  100000 non-null  object             
 3   purchase_amount             100000 non-null  float64            
 4   is_reordered                100000 non-null  int64              
 5   event_time                  100000 non-null  datetime64[ns, UTC]
 6   n_days_since_last_purchase  100000 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(2), int64(1), object(3)
memory usage: 5.3+ MB


In [40]:
# Load Feature definitions
customers_feature_group.load_feature_definitions(data_frame = customer_data)

[FeatureDefinition(feature_name='customer_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='sex', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='is_married', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='age_18-29', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='age_30-39', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='age_40-49', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='age_50-59', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='age_60-69

In [41]:
# Load Feature definitions
orders_feature_group.load_feature_definitions(data_frame = orders_data)

[FeatureDefinition(feature_name='order_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='customer_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='product_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='purchase_amount', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='is_reordered', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='n_days_since_last_purchase', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None)]

In [42]:
customers_feature_group.create(s3_uri = f"s3://{s3_bucket_name}/{prefix}",
                               record_identifier_name=record_identifier_feature_name, event_time_feature_name="event_time",
                                role_arn=role,
                                enable_online_store=True)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:866824485776:feature-group/customers-fg-02-13-01-25',
 'ResponseMetadata': {'RequestId': 'bd641334-f38b-47ec-84ef-cc99833666ba',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bd641334-f38b-47ec-84ef-cc99833666ba',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '101',
   'date': 'Thu, 02 May 2024 13:06:03 GMT'},
  'RetryAttempts': 0}}

In [43]:
orders_feature_group.create(s3_uri = f"s3://{s3_bucket_name}/{prefix}",
                               record_identifier_name=record_identifier_feature_name, event_time_feature_name="event_time",
                                role_arn=role,
                                enable_online_store=True)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:866824485776:feature-group/orders-fg-02-13-01-25',
 'ResponseMetadata': {'RequestId': 'eb2e5a9c-aba7-43bd-9ffc-40f10fc10e5d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'eb2e5a9c-aba7-43bd-9ffc-40f10fc10e5d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '98',
   'date': 'Thu, 02 May 2024 13:06:49 GMT'},
  'RetryAttempts': 0}}

In [44]:
customers_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:866824485776:feature-group/customers-fg-02-13-01-25',
 'FeatureGroupName': 'customers-fg-02-13-01-25',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'customer_id',
   'FeatureType': 'String'},
  {'FeatureName': 'sex', 'FeatureType': 'Integral'},
  {'FeatureName': 'is_married', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'String'},
  {'FeatureName': 'age_18-29', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_30-39', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_40-49', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_50-59', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_60-69', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_70-plus', 'FeatureType': 'Integral'},
  {'FeatureName': 'n_days_active', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2024, 5, 2, 13, 6, 3, 461000, tzinfo=tzlocal()),
 

In [45]:
orders_feature_group.describe() 

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:866824485776:feature-group/orders-fg-02-13-01-25',
 'FeatureGroupName': 'orders-fg-02-13-01-25',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'order_id', 'FeatureType': 'String'},
  {'FeatureName': 'customer_id', 'FeatureType': 'String'},
  {'FeatureName': 'product_id', 'FeatureType': 'String'},
  {'FeatureName': 'purchase_amount', 'FeatureType': 'Fractional'},
  {'FeatureName': 'is_reordered', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'String'},
  {'FeatureName': 'n_days_since_last_purchase', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2024, 5, 2, 13, 6, 49, 748000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://sagemaker-us-east-1-866824485776/aws-mlops-sagemaker',
   'ResolvedOutputS3Uri': 's3://sagemaker-us-east-1-86682

In [46]:
# Add metadata to a feature
# description and parameters

from sagemaker.feature_store.inputs import FeatureParameter

customers_feature_group.update_feature_metadata(feature_name = "customer_id", description= "The ID of the customer, it is also part of Order feature group",
                                                parameter_additions=[FeatureParameter("idType","primarykey")])

{'ResponseMetadata': {'RequestId': 'dc38931a-ad9d-4d67-a84b-d9027ce3207d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dc38931a-ad9d-4d67-a84b-d9027ce3207d',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 02 May 2024 13:13:30 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [47]:
customers_feature_group.describe_feature_metadata("customer_id")

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:866824485776:feature-group/customers-fg-02-13-01-25',
 'FeatureGroupName': 'customers-fg-02-13-01-25',
 'FeatureName': 'customer_id',
 'FeatureType': 'String',
 'CreationTime': datetime.datetime(2024, 5, 2, 13, 6, 3, 461000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 5, 2, 13, 13, 30, 993000, tzinfo=tzlocal()),
 'Description': 'The ID of the customer, it is also part of Order feature group',
 'Parameters': [{'Key': 'idType', 'Value': 'primarykey'}],
 'ResponseMetadata': {'RequestId': 'f0fa0936-c681-447a-92d8-1c548c0c2a7d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f0fa0936-c681-447a-92d8-1c548c0c2a7d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '398',
   'date': 'Thu, 02 May 2024 13:13:58 GMT'},
  'RetryAttempts': 0}}

In [52]:
sagemaker_session.boto_session.client("sagemaker", region_name=region).search(
    Resource="FeatureMetadata",
    SearchExpression={
        "Filters":[
            {
                "Name": "FeatureGroupName",
                "Operator": "Contains",
                "Value": "customers-fg"
            },
            {"Name": "Parameters.idType", "Operator": "Equals", "Value": "primarykey"}
        ]
    })

{'Results': [{'FeatureMetadata': {'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:866824485776:feature-group/customers-fg-02-13-01-25',
    'FeatureGroupName': 'customers-fg-02-13-01-25',
    'FeatureName': 'customer_id',
    'FeatureType': 'String',
    'CreationTime': datetime.datetime(2024, 5, 2, 13, 6, 3, tzinfo=tzlocal()),
    'LastModifiedTime': datetime.datetime(2024, 5, 2, 13, 13, 30, tzinfo=tzlocal()),
    'Description': 'The ID of the customer, it is also part of Order feature group',
    'Parameters': [{'Key': 'idType', 'Value': 'primarykey'}]}}],
 'ResponseMetadata': {'RequestId': '58c2749c-ccd0-4a10-89db-7d5e816667b7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '58c2749c-ccd0-4a10-89db-7d5e816667b7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '461',
   'date': 'Thu, 02 May 2024 13:18:45 GMT'},
  'RetryAttempts': 0}}

# Ingest the data to the Feature Group

In [53]:
customers_feature_group.ingest(data_frame = customer_data, max_workers=3,wait=True)

IngestionManagerPandas(feature_group_name='customers-fg-02-13-01-25', feature_definitions={'customer_id': {'FeatureName': 'customer_id', 'FeatureType': 'String'}, 'sex': {'FeatureName': 'sex', 'FeatureType': 'Integral'}, 'is_married': {'FeatureName': 'is_married', 'FeatureType': 'Integral'}, 'event_time': {'FeatureName': 'event_time', 'FeatureType': 'String'}, 'age_18-29': {'FeatureName': 'age_18-29', 'FeatureType': 'Integral'}, 'age_30-39': {'FeatureName': 'age_30-39', 'FeatureType': 'Integral'}, 'age_40-49': {'FeatureName': 'age_40-49', 'FeatureType': 'Integral'}, 'age_50-59': {'FeatureName': 'age_50-59', 'FeatureType': 'Integral'}, 'age_60-69': {'FeatureName': 'age_60-69', 'FeatureType': 'Integral'}, 'age_70-plus': {'FeatureName': 'age_70-plus', 'FeatureType': 'Integral'}, 'n_days_active': {'FeatureName': 'n_days_active', 'FeatureType': 'Fractional'}}, sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f5caed696c0>, sagemaker_session=<sagemaker.session.Session o

In [54]:
orders_feature_group.ingest(data_frame = orders_data, max_workers=3,wait=True)

IngestionManagerPandas(feature_group_name='orders-fg-02-13-01-25', feature_definitions={'order_id': {'FeatureName': 'order_id', 'FeatureType': 'String'}, 'customer_id': {'FeatureName': 'customer_id', 'FeatureType': 'String'}, 'product_id': {'FeatureName': 'product_id', 'FeatureType': 'String'}, 'purchase_amount': {'FeatureName': 'purchase_amount', 'FeatureType': 'Fractional'}, 'is_reordered': {'FeatureName': 'is_reordered', 'FeatureType': 'Integral'}, 'event_time': {'FeatureName': 'event_time', 'FeatureType': 'String'}, 'n_days_since_last_purchase': {'FeatureName': 'n_days_since_last_purchase', 'FeatureType': 'Fractional'}}, sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f5caed696c0>, sagemaker_session=<sagemaker.session.Session object at 0x7f5caea7a6e0>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7f5cab3023b0>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

In [56]:
customer_id = "C1"
sample_record = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).get_record(
    FeatureGroupName=customers_feature_group_name, RecordIdentifierValueAsString=str(customer_id)
)

In [57]:
sample_record

{'ResponseMetadata': {'RequestId': '6ba01a11-54fe-4d00-a08e-1c06681f9ce5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6ba01a11-54fe-4d00-a08e-1c06681f9ce5',
   'content-type': 'application/json',
   'content-length': '877',
   'date': 'Thu, 02 May 2024 13:32:28 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'customer_id', 'ValueAsString': 'C1'},
  {'FeatureName': 'sex', 'ValueAsString': '0'},
  {'FeatureName': 'is_married', 'ValueAsString': '1'},
  {'FeatureName': 'event_time', 'ValueAsString': '2024-05-02T05:39:10.965Z'},
  {'FeatureName': 'age_18-29', 'ValueAsString': '0'},
  {'FeatureName': 'age_30-39', 'ValueAsString': '0'},
  {'FeatureName': 'age_40-49', 'ValueAsString': '0'},
  {'FeatureName': 'age_50-59', 'ValueAsString': '0'},
  {'FeatureName': 'age_60-69', 'ValueAsString': '0'},
  {'FeatureName': 'age_70-plus', 'ValueAsString': '1'},
  {'FeatureName': 'n_days_active', 'ValueAsString': '0.2034246575342466'}]}

In [58]:
all_records = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": customers_feature_group_name,
            "RecordIdentifiersValueAsString": ["C1","C2"],
        },
        {
            "FeatureGroupName": orders_feature_group_name,
            "RecordIdentifiersValueAsString": ["C1","C2"],
        },
    ]
)

In [59]:
all_records

{'ResponseMetadata': {'RequestId': '6c15f5c0-9145-4084-ba9e-44b02d9c7663',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6c15f5c0-9145-4084-ba9e-44b02d9c7663',
   'content-type': 'application/json',
   'content-length': '3408',
   'date': 'Thu, 02 May 2024 13:33:33 GMT'},
  'RetryAttempts': 0},
 'Records': [{'FeatureGroupName': 'customers-fg-02-13-01-25',
   'RecordIdentifierValueAsString': 'C1',
   'Record': [{'FeatureName': 'customer_id', 'ValueAsString': 'C1'},
    {'FeatureName': 'sex', 'ValueAsString': '0'},
    {'FeatureName': 'is_married', 'ValueAsString': '1'},
    {'FeatureName': 'event_time', 'ValueAsString': '2024-05-02T05:39:10.965Z'},
    {'FeatureName': 'age_18-29', 'ValueAsString': '0'},
    {'FeatureName': 'age_30-39', 'ValueAsString': '0'},
    {'FeatureName': 'age_40-49', 'ValueAsString': '0'},
    {'FeatureName': 'age_50-59', 'ValueAsString': '0'},
    {'FeatureName': 'age_60-69', 'ValueAsString': '0'},
    {'FeatureName': 'age_70-plus', 'ValueAsStr

In [61]:
# Add features to feature group
from sagemaker.feature_store.feature_definition import StringFeatureDefinition

customers_feature_group.update(feature_additions=[StringFeatureDefinition("email")])

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:866824485776:feature-group/customers-fg-02-13-01-25',
 'ResponseMetadata': {'RequestId': 'b5c8a73d-3934-499b-b36f-4d7b79754f40',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b5c8a73d-3934-499b-b36f-4d7b79754f40',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '101',
   'date': 'Thu, 02 May 2024 13:35:46 GMT'},
  'RetryAttempts': 0}}

In [64]:
customers_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:866824485776:feature-group/customers-fg-02-13-01-25',
 'FeatureGroupName': 'customers-fg-02-13-01-25',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'customer_id',
   'FeatureType': 'String'},
  {'FeatureName': 'sex', 'FeatureType': 'Integral'},
  {'FeatureName': 'is_married', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'String'},
  {'FeatureName': 'age_18-29', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_30-39', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_40-49', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_50-59', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_60-69', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_70-plus', 'FeatureType': 'Integral'},
  {'FeatureName': 'n_days_active', 'FeatureType': 'Fractional'},
  {'FeatureName': 'email', 'FeatureType': 'String'}],
 'CreationTime': datetime.dateti

In [71]:
# Updated customers
customer_data_updated = pd.read_csv("transformed/customers_updated.csv")
customer_data_updated.head()

,customer_id,sex,is_married,event_time,email,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C10001,0,1,2024-05-02T13:43:28.295Z,kennethday@example.net,0,0,0,0,0,1,0.345890
1,C10002,0,1,2024-05-02T13:43:28.296Z,smelendez@example.org,0,0,0,0,0,1,0.884247
2,C10003,1,0,2024-05-02T13:43:28.298Z,brianperry@example.net,0,1,0,0,0,0,0.734247
3,C10004,0,0,2024-05-02T13:43:28.300Z,alyssamorgan@example.net,0,0,0,0,1,0,0.386301
4,C10005,0,1,2024-05-02T13:43:28.301Z,cjenkins@example.org,0,0,0,0,1,0,0.363699


In [72]:
customers_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:866824485776:feature-group/customers-fg-02-13-01-25',
 'FeatureGroupName': 'customers-fg-02-13-01-25',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'customer_id',
   'FeatureType': 'String'},
  {'FeatureName': 'sex', 'FeatureType': 'Integral'},
  {'FeatureName': 'is_married', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'String'},
  {'FeatureName': 'age_18-29', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_30-39', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_40-49', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_50-59', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_60-69', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_70-plus', 'FeatureType': 'Integral'},
  {'FeatureName': 'n_days_active', 'FeatureType': 'Fractional'},
  {'FeatureName': 'email', 'FeatureType': 'String'}],
 'CreationTime': datetime.dateti

In [73]:
customer_data_updated.columns

Index(['customer_id', 'sex', 'is_married', 'event_time', 'email', 'age_18-29',
       'age_30-39', 'age_40-49', 'age_50-59', 'age_60-69', 'age_70-plus',
       'n_days_active'],
      dtype='object')

In [74]:
customers_feature_group.ingest(data_frame = customer_data_updated, max_workers=3,wait=True)

IngestionManagerPandas(feature_group_name='customers-fg-02-13-01-25', feature_definitions={'customer_id': {'FeatureName': 'customer_id', 'FeatureType': 'String'}, 'sex': {'FeatureName': 'sex', 'FeatureType': 'Integral'}, 'is_married': {'FeatureName': 'is_married', 'FeatureType': 'Integral'}, 'event_time': {'FeatureName': 'event_time', 'FeatureType': 'String'}, 'age_18-29': {'FeatureName': 'age_18-29', 'FeatureType': 'Integral'}, 'age_30-39': {'FeatureName': 'age_30-39', 'FeatureType': 'Integral'}, 'age_40-49': {'FeatureName': 'age_40-49', 'FeatureType': 'Integral'}, 'age_50-59': {'FeatureName': 'age_50-59', 'FeatureType': 'Integral'}, 'age_60-69': {'FeatureName': 'age_60-69', 'FeatureType': 'Integral'}, 'age_70-plus': {'FeatureName': 'age_70-plus', 'FeatureType': 'Integral'}, 'n_days_active': {'FeatureName': 'n_days_active', 'FeatureType': 'Fractional'}, 'email': {'FeatureName': 'email', 'FeatureType': 'String'}}, sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7

In [75]:
customer_id = "C10001"
sample_record = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).get_record(
    FeatureGroupName=customers_feature_group_name, RecordIdentifierValueAsString=str(customer_id)
)

In [76]:
sample_record

{'ResponseMetadata': {'RequestId': 'aa3adfed-fe11-473f-ae76-135e4ee1c26f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'aa3adfed-fe11-473f-ae76-135e4ee1c26f',
   'content-type': 'application/json',
   'content-length': '970',
   'date': 'Thu, 02 May 2024 13:46:31 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'customer_id', 'ValueAsString': 'C10001'},
  {'FeatureName': 'sex', 'ValueAsString': '0'},
  {'FeatureName': 'is_married', 'ValueAsString': '1'},
  {'FeatureName': 'event_time', 'ValueAsString': '2024-05-02T13:43:28.295Z'},
  {'FeatureName': 'age_18-29', 'ValueAsString': '0'},
  {'FeatureName': 'age_30-39', 'ValueAsString': '0'},
  {'FeatureName': 'age_40-49', 'ValueAsString': '0'},
  {'FeatureName': 'age_50-59', 'ValueAsString': '0'},
  {'FeatureName': 'age_60-69', 'ValueAsString': '0'},
  {'FeatureName': 'age_70-plus', 'ValueAsString': '1'},
  {'FeatureName': 'n_days_active', 'ValueAsString': '0.345890410958904'},
  {'FeatureName': 'email', 'ValueAs